<a href="https://colab.research.google.com/github/chengulatj/arrowhead-traffic-analysis/blob/main/Chiefs_Bill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pykml
!pip install fastkml
!pip install polyline
!pip install pytz haversine requests polyline pandas folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import pandas as pd
import folium

# Paths to your files
kmz_cams_file = "/content/drive/MyDrive/cams.kmz"  # Path to your KMZ file

# Extract the KML file from the KMZ file
kml_extracted_path = "/content/drive/MyDrive/extracted_kml.kml"
with ZipFile(kmz_cams_file, 'r') as kmz:
    # Extract the KML file
    kml_filename = [file for file in kmz.namelist() if file.endswith('.kml')][0]
    kmz.extract(kml_filename, path="/content/drive/MyDrive")
    kml_full_path = f"/content/drive/MyDrive/{kml_filename}"

# Parse the extracted KML file to retrieve placemark data
tree = ET.parse(kml_full_path)
root = tree.getroot()

# Define the namespace to parse KML properly
namespace = {"kml": "http://www.opengis.net/kml/2.2"}

# Extract placemark names and coordinates
camera_data = []
for placemark in root.findall(".//kml:Placemark", namespace):
    name = placemark.find("kml:name", namespace).text
    coords = placemark.find(".//kml:coordinates", namespace).text.strip()
    lon, lat, _ = map(float, coords.split(","))
    camera_data.append({"Name": name, "Latitude": lat, "Longitude": lon})

# Convert to a DataFrame
camera_df = pd.DataFrame(camera_data)

# Remove rows where 'Name' contains "Arrowhead" since that's the actual destination
camera_df = camera_df[~camera_df["Name"].str.contains("|".join(["Arrowhead", "North", "East", "West", "South"]), case=False, na=False)]

# Display the Camera DataFrame
print("Camera  Locations:")
print(camera_df)


Camera  Locations:
                              Name   Latitude  Longitude
0   I-70 EB Past Blue Ridge Cutoff  39.047970 -94.466341
1              I-70 EB @ US-40 Hwy  39.047025 -94.444162
2       I-70 EB at Blue Ridge Blvd  39.046588 -94.437408
3               I-70 EB at 31st St  39.067333 -94.509992
4           I-70 WB Past Noland Rd  39.046587 -94.424266
5           I-70 EB Past Noland Rd  39.045194 -94.399116
6       I-70 EB at Lee's Summit Rd  39.044593 -94.386910
8              I-435 SB at 23rd St  39.081620 -94.491118
9      I-70 WB Past Van Brunt Blvd  39.072746 -94.526543
10            I-70 EB Before I-470  39.043612 -94.377263
11          I-435 SB at Raytown Rd  39.050540 -94.499744
13          I-435 SB at Stadium Dr  39.055722 -94.494419
14   I-70 EB at SE Corner of I-435  39.057050 -94.487158
15           I-70 WB at Stadium Dr  39.055973 -94.481648
16               I-435 NB at US_40  39.062850 -94.487709
17      I-70 EB at Manchester Tfwy  39.063150 -94.500015
18   I-70 EB

In [ ]:
# Paths to your files
kmz_dms_file = "/content/drive/MyDrive/dms.kmz"  # Path to your KMZ file

# Extract the KML file from the KMZ file
kml_extracted_path = "/content/drive/MyDrive/extracted_kml.kml"
with ZipFile(kmz_dms_file, 'r') as kmz:
    # Extract the KML file
    kml_filename = [file for file in kmz.namelist() if file.endswith('.kml')][0]
    kmz.extract(kml_filename, path="/content/drive/MyDrive")
    kml_full_path = f"/content/drive/MyDrive/{kml_filename}"

# Parse the extracted KML file to retrieve placemark data
tree = ET.parse(kml_full_path)
root = tree.getroot()

# Define the namespace to parse KML properly
namespace = {"kml": "http://www.opengis.net/kml/2.2"}

# Extract placemark names and coordinates
dms_data = []
for placemark in root.findall(".//kml:Placemark", namespace):
    name = placemark.find("kml:name", namespace).text
    coords = placemark.find(".//kml:coordinates", namespace).text.strip()
    lon, lat, _ = map(float, coords.split(","))
    dms_data.append({"Name": name, "Latitude": lat, "Longitude": lon})

# Convert to a DataFrame
dms_df = pd.DataFrame(dms_data)

# Display the Camera and DMS DataFrame
print("DMS Locations:")
print(dms_df)


DMS Locations:
                              Name   Latitude  Longitude
0        I-435 NB @ Eastwood TFWY   39.032335 -94.500228
1   I-170 EB @ Before Sterling ave  39.047574 -94.457909
2     I-170 WB @ Before Blue Ridge  39.047839 -94.454545
3    I70 WB @ Before 40/Blue Ridge  39.046683 -94.425800
4    I70 EB @ Before Lee Summit Rd  39.045510 -94.407959
5         I-435 SB @ Before 40 Hwy  39.072685 -94.488767
6     I-70 WB @ Before Jackson ave  39.071902 -94.522787
7  I-70 EB at Before 40/Manchester  39.069813 -94.514621


In [ ]:
import os
import folium
import requests
import pandas as pd
from polyline import decode as decode_polyline

# Function to load the API key securely from Google Drive
def load_api_key(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            if key == 'api_key':
                return value

# Fetch route details from Google Maps Directions API
def fetch_routes(api_key, origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin,
        "destination": destination,
        "key": api_key,
        "alternatives": "true"  # Enable fetching alternative routes
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            # Extract all routes
            routes = []
            for route in data['routes']:
                polyline = route['overview_polyline']['points']
                routes.append(decode_polyline(polyline))  # Decode the polyline into lat/lng points
            return routes
        else:
            print(f"Error from API: {data['status']}")
    else:
        print(f"HTTP Error: {response.status_code}")
    return None

# Plot routes on a map
def plot_routes_on_map(api_key, data):
    # Initialize the map
    m = folium.Map(location=[39.048786, -94.484566], zoom_start=13)
    colors = ['blue', 'green', 'purple', 'orange', 'red']  # Colors for multiple routes

    for index, row in data.iterrows():
        origin = f"{row['origin_lat']},{row['origin_lng']}"
        destination = f"{row['destination_lat']},{row['destination_lng']}"

        # Fetch all routes data
        routes = fetch_routes(api_key, origin, destination)
        if routes:
            for i, route_coords in enumerate(routes):
                # Cycle through colors if there are more routes than colors
                color = colors[i % len(colors)]
                # Add the real route to the map
                folium.PolyLine(route_coords, color=color, weight=2.5, opacity=0.7,
                                tooltip=f"Route {i + 1}").add_to(m)

            # Add markers for origin and destination
            folium.Marker(
                location=[row['origin_lat'], row['origin_lng']],
                popup=f"Origin: {row['origin']}",
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(m)

            folium.Marker(
                location=[row['destination_lat'], row['destination_lng']],
                popup=f"Destination: {row['destination']}",
                icon=folium.Icon(color='red', icon='info-sign')
            ).add_to(m)

    return m

# Main script
if __name__ == "__main__":
    # Path to the file containing your Google Maps API key on Google Drive
    api_key_path = "/content/drive/MyDrive/API_K.txt"  # Update with your actual file path

    # Load the API key
    api_key = load_api_key(api_key_path)

    # Load the dataset (replace with the actual dataset path)
    dataset_path = "/content/drive/MyDrive/chiefs-bills.xlsx" # Update with your dataset path
    df = pd.read_excel(dataset_path)

    # Parse the `query` column to extract origin and destination coordinates
    df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = df['query'].str.split(', ', expand=True)
    df['origin_lat'] = df['origin_lat'].astype(float)
    df['origin_lng'] = df['origin_lng'].astype(float)
    df['destination_lat'] = df['destination_lat'].astype(float)
    df['destination_lng'] = df['destination_lng'].astype(float)

    # Call the function to plot the routes on a map
    traffic_map = plot_routes_on_map(api_key, df)

    # Save the map to an HTML file
    traffic_map.save("traffic_routes_map24.html")
    print("Map saved as traffic_routes_map24.html")


Map saved as traffic_routes_map24.html


In [ ]:
import os
import folium
import requests
import pandas as pd
from polyline import decode as decode_polyline
from datetime import datetime, timedelta
from folium.plugins import HeatMap

# Function to load the API key securely from Google Drive
def load_api_key(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            if key == 'api_key':
                return value

# Convert UTC time to Central Time (CT)
def convert_to_central_time(utc_time):
    # If the input is an integer (Unix timestamp)
    if isinstance(utc_time, int):
        utc_dt = datetime.utcfromtimestamp(utc_time)  # Convert Unix timestamp to datetime
    else:  # If the input is a string
        utc_dt = datetime.strptime(utc_time, "%Y-%m-%d %H:%M:%S")

    # Convert to Central Time (UTC-6)
    central_dt = utc_dt - timedelta(hours=6)
    return central_dt.strftime("%Y-%m-%d %H:%M:%S")

# Fetch route details from Google Maps Directions API
def fetch_routes(api_key, origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin,
        "destination": destination,
        "key": api_key,
        "alternatives": "true"  # Enable fetching alternative routes
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            # Extract all routes
            routes = []
            for route in data['routes']:
                polyline = route['overview_polyline']['points']
                routes.append(decode_polyline(polyline))  # Decode the polyline into lat/lng points
            return routes
        else:
            print(f"Error from API: {data['status']}")
    else:
        print(f"HTTP Error: {response.status_code}")
    return None

# Create a heatmap based on routes and time intervals
def create_route_heatmap(data, api_key, time_interval):
    # Filter data by the specified time interval
    filtered_data = data[data['central_time'].str.contains('|'.join(time_interval))]

    # Initialize the map
    m = folium.Map(location=[39.048786, -94.484566], zoom_start=13)

    # Extract route coordinates for the heatmap
    heat_data = []
    for index, row in filtered_data.iterrows():
        origin = f"{row['origin_lat']},{row['origin_lng']}"
        destination = f"{row['destination_lat']},{row['destination_lng']}"

        # Fetch all routes data
        routes = fetch_routes(api_key, origin, destination)
        if routes:
            for route_coords in routes:
                for lat, lng in route_coords:
                    heat_data.append([lat, lng, 1])  # Add each point with a weight of 1

    # Add HeatMap layer
    HeatMap(heat_data).add_to(m)

    return m

# Main script
if __name__ == "__main__":
    # Path to the file containing your Google Maps API key on Google Drive
    api_key_path = "/content/drive/MyDrive/API_K.txt"  # Update with your actual file path

    # Load the API key
    api_key = load_api_key(api_key_path)

    # Load the dataset (replace with the actual dataset path)
    dataset_path = "/content/drive/MyDrive/chiefs-bills.xlsx"  # Update with your dataset path
    df = pd.read_excel(dataset_path)

    # Parse the `query` column to extract origin and destination coordinates
    df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = df['query'].str.split(', ', expand=True)
    df['origin_lat'] = df['origin_lat'].astype(float)
    df['origin_lng'] = df['origin_lng'].astype(float)
    df['destination_lat'] = df['destination_lat'].astype(float)
    df['destination_lng'] = df['destination_lng'].astype(float)

    # Convert UTC timestamps to Central Time if a timestamp column exists
    if 'timestamp' in df.columns:
        df['central_time'] = df['timestamp'].apply(convert_to_central_time)

    # Define the time interval for analysis
    time_interval = "12:30", "14:30"


    # Create the heatmap for the specified time interval
    heatmap = create_route_heatmap(df, api_key, time_interval)

    # Save the map to an HTML file
    heatmap.save("traffic_route_heatmap2.html")
    print("Map saved as traffic_route_heatmap2.html")


Map saved as traffic_route_heatmap2.html


In [ ]:
import os
import folium
import requests
import pandas as pd
from polyline import decode as decode_polyline
from datetime import datetime, timedelta
from folium.plugins import HeatMap

# Function to load the API key securely from Google Drive
def load_api_key(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            if key == 'api_key':
                return value

# Convert UTC time to Central Time (CT)
def convert_to_central_time(utc_time):
    # If the input is an integer (Unix timestamp)
    if isinstance(utc_time, int):
        utc_dt = datetime.utcfromtimestamp(utc_time)  # Convert Unix timestamp to datetime
    else:  # If the input is a string
        utc_dt = datetime.strptime(utc_time, "%Y-%m-%d %H:%M:%S")

    # Convert to Central Time (UTC-6)
    central_dt = utc_dt - timedelta(hours=6)
    return central_dt.strftime("%Y-%m-%d %H:%M:%S")

# Fetch route details from Google Maps Directions API
def fetch_routes(api_key, origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin,
        "destination": destination,
        "key": api_key,
        "alternatives": "true"  # Enable fetching alternative routes
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            # Extract all routes
            routes = []
            for route in data['routes']:
                polyline = route['overview_polyline']['points']
                routes.append(decode_polyline(polyline))  # Decode the polyline into lat/lng points
            return routes
        else:
            print(f"Error from API: {data['status']}")
    else:
        print(f"HTTP Error: {response.status_code}")
    return None

# Create a heatmap based on routes and time intervals
def create_route_heatmap(data, api_key, time_interval):
    # Filter data by the specified time interval
    filtered_data = data[data['central_time'].str.contains('|'.join(time_interval))]

    # Initialize the map
    m = folium.Map(location=[39.048786, -94.484566], zoom_start=13)

    # Extract route coordinates for the heatmap
    heat_data = []
    for index, row in filtered_data.iterrows():
        origin = f"{row['origin_lat']},{row['origin_lng']}"
        destination = f"{row['destination_lat']},{row['destination_lng']}"

        # Fetch all routes data
        routes = fetch_routes(api_key, origin, destination)
        if routes:
            for route_coords in routes:
                for lat, lng in route_coords:
                    heat_data.append([lat, lng, 1])  # Add each point with a weight of 1

    # Add HeatMap layer
    HeatMap(heat_data).add_to(m)

    return m

# Main script
if __name__ == "__main__":
    # Path to the file containing your Google Maps API key on Google Drive
    api_key_path = "/content/drive/MyDrive/API_K.txt"  # Update with your actual file path

    # Load the API key
    api_key = load_api_key(api_key_path)

    # Load the dataset (replace with the actual dataset path)
    dataset_path = "/content/drive/MyDrive/chiefs-bills.xlsx"  # Update with your dataset path
    df = pd.read_excel(dataset_path)

    # Parse the `query` column to extract origin and destination coordinates
    df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = df['query'].str.split(', ', expand=True)
    df['origin_lat'] = df['origin_lat'].astype(float)
    df['origin_lng'] = df['origin_lng'].astype(float)
    df['destination_lat'] = df['destination_lat'].astype(float)
    df['destination_lng'] = df['destination_lng'].astype(float)

    # Convert UTC timestamps to Central Time if a timestamp column exists
    if 'timestamp' in df.columns:
        df['central_time'] = df['timestamp'].apply(convert_to_central_time)

    # Define the time interval for analysis
    time_interval = "14:30", "16:30"


    # Create the heatmap for the specified time interval
    heatmap = create_route_heatmap(df, api_key, time_interval)

    # Save the map to an HTML file
    heatmap.save("traffic_route_heatmap3.html")
    print("Map saved as traffic_route_heatmap3.html")


Map saved as traffic_route_heatmap3.html


In [ ]:
import os
import folium
import requests
import pandas as pd
from polyline import decode as decode_polyline
from datetime import datetime, timedelta
from folium.plugins import HeatMap

# Function to load the API key securely from Google Drive
def load_api_key(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            if key == 'api_key':
                return value

# Convert UTC time to Central Time (CT)
def convert_to_central_time(utc_time):
    # If the input is an integer (Unix timestamp)
    if isinstance(utc_time, int):
        utc_dt = datetime.utcfromtimestamp(utc_time)  # Convert Unix timestamp to datetime
    else:  # If the input is a string
        utc_dt = datetime.strptime(utc_time, "%Y-%m-%d %H:%M:%S")

    # Convert to Central Time (UTC-6)
    central_dt = utc_dt - timedelta(hours=6)
    return central_dt.strftime("%Y-%m-%d %H:%M:%S")

# Fetch route details from Google Maps Directions API
def fetch_routes(api_key, origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin,
        "destination": destination,
        "key": api_key,
        "alternatives": "true"  # Enable fetching alternative routes
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            # Extract all routes
            routes = []
            for route in data['routes']:
                polyline = route['overview_polyline']['points']
                routes.append(decode_polyline(polyline))  # Decode the polyline into lat/lng points
            return routes
        else:
            print(f"Error from API: {data['status']}")
    else:
        print(f"HTTP Error: {response.status_code}")
    return None

# Create a heatmap based on routes and time intervals
def create_route_heatmap(data, api_key, time_interval):
    # Filter data by the specified time interval
    filtered_data = data[data['central_time'].str.contains('|'.join(time_interval))]

    # Initialize the map
    m = folium.Map(location=[39.048786, -94.484566], zoom_start=13)

    # Extract route coordinates for the heatmap
    heat_data = []
    for index, row in filtered_data.iterrows():
        origin = f"{row['origin_lat']},{row['origin_lng']}"
        destination = f"{row['destination_lat']},{row['destination_lng']}"

        # Fetch all routes data
        routes = fetch_routes(api_key, origin, destination)
        if routes:
            for route_coords in routes:
                for lat, lng in route_coords:
                    heat_data.append([lat, lng, 1])  # Add each point with a weight of 1

    # Add HeatMap layer
    HeatMap(heat_data).add_to(m)

    return m

# Main script
if __name__ == "__main__":
    # Path to the file containing your Google Maps API key on Google Drive
    api_key_path = "/content/drive/MyDrive/API_K.txt"  # Update with your actual file path

    # Load the API key
    api_key = load_api_key(api_key_path)

    # Load the dataset (replace with the actual dataset path)
    dataset_path = "/content/drive/MyDrive/chiefs-bills.xlsx"  # Update with your dataset path
    df = pd.read_excel(dataset_path)

    # Parse the `query` column to extract origin and destination coordinates
    df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = df['query'].str.split(', ', expand=True)
    df['origin_lat'] = df['origin_lat'].astype(float)
    df['origin_lng'] = df['origin_lng'].astype(float)
    df['destination_lat'] = df['destination_lat'].astype(float)
    df['destination_lng'] = df['destination_lng'].astype(float)

    # Convert UTC timestamps to Central Time if a timestamp column exists
    if 'timestamp' in df.columns:
        df['central_time'] = df['timestamp'].apply(convert_to_central_time)

    # Define the time interval for analysis
    time_interval = "16:30", "17:30"


    # Create the heatmap for the specified time interval
    heatmap = create_route_heatmap(df, api_key, time_interval)

    # Save the map to an HTML file
    heatmap.save("traffic_route_heatmap4.html")
    print("Map saved as traffic_route_heatmap4.html")


Map saved as traffic_route_heatmap4.html


In [ ]:
import os
import folium
import requests
import pandas as pd
from polyline import decode as decode_polyline
from pytz import timezone
from datetime import datetime
from haversine import haversine, Unit
import json


# Function to load the API key securely from Google Drive
def load_api_key(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            if key == 'api_key':
                return value

# Fetch route details from Google Maps Directions API
def fetch_route_polyline(api_key, origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin,
        "destination": destination,
        "key": api_key,
        "alternatives": "true"  # Enable fetching alternative routes
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            return data['routes'][0]['overview_polyline']['points']  # Return the encoded polyline
        else:
            print(f"Error from API: {data['status']}")
    else:
        print(f"HTTP Error: {response.status_code}")
    return None


# Convert UTC times to Central Time
def convert_to_central_time(utc_time_str):
    utc = timezone('UTC')
    central = timezone('US/Central')
    utc_time = datetime.strptime(utc_time_str, "%m/%d/%Y %H:%M:%S")
    central_time = utc.localize(utc_time).astimezone(central)
    return central_time.strftime("%m/%d/%Y %H:%M:%S")

# Analyze congestion flags
def update_congestion_flags(speeds_mph):
    return [speed < 20 if speed is not None else False for speed in speeds_mph]

# Find lat/long of congested points based on distance along polyline
def get_congestion_coordinates(route_polyline, distances_miles):
    route_coords = decode_polyline(route_polyline)
    congestion_coords = []
    total_distance = 0

    for i in range(1, len(route_coords)):
        # Calculate segment distance in miles
        dist = haversine(route_coords[i - 1], route_coords[i])
        total_distance += dist

        # Match congestion distances to coordinates
        for distance in distances_miles:
            if total_distance >= distance:
                congestion_coords.append(route_coords[i])
                distances_miles.remove(distance)
                break
        if not distances_miles:
            break
    return congestion_coords

# Main script
if __name__ == "__main__":
    # Set file paths
    api_key_path = "/content/drive/MyDrive/API_K.txt"  # directions API
    dataset_path = "/content/drive/MyDrive/chiefs-bills.xlsx"

    # Load the API key
    api_key = load_api_key(api_key_path)

    # Load the dataset
    df = pd.read_excel(dataset_path)

    # Parse and extract route polylines for all rows
    df['origin'] = df['origin_coordinates']
    df['destination'] = df['destination_coordinates']
    df['polyline'] = df.apply(
        lambda row: fetch_route_polyline(api_key, row['origin'], row['destination']), axis=1
    )

    # Parse road timing data
    df['road_timing_parsed'] = df['road_distance_timing(meters:seconds)'].apply(
        lambda x: json.loads(x.replace("'", '"'))
    )

    # Convert road timing data and calculate speeds safely
    df['speeds_mph'], df['congestion_flags_20mph'] = zip(
    *df['road_timing_parsed'].apply(
        lambda timing: (
            [pd.to_numeric(dist, errors='coerce') * 0.000621371 / (pd.to_numeric(time, errors='coerce') / 3600)
             if pd.to_numeric(time, errors='coerce') > 0 else None
             for dist, time in timing.items()],
            [pd.to_numeric(dist, errors='coerce') * 0.000621371 / (pd.to_numeric(time, errors='coerce') / 3600) < 40
             if pd.to_numeric(time, errors='coerce') > 0 else False
             for dist, time in timing.items()]
        )
    )
)

  # Extract congestion details
    df['congestion_details'] = df.apply(
    lambda row: [
        (convert_to_central_time(row['datetime_utc']), pd.to_numeric(dist, errors='coerce') * 0.000621371)
        for dist, flag in zip(row['road_timing_parsed'].keys(), row['congestion_flags_20mph'])
        if flag and pd.to_numeric(dist, errors='coerce') is not None
    ],
    axis=1
)

    # Find coordinates for congestion points
    df['congestion_coords'] = df.apply(
        lambda row: get_congestion_coordinates(row['polyline'], [dist for _, dist in row['congestion_details']]),
        axis=1
    )

  # Map congestion points
    congestion_map = folium.Map(location=[39.048786, -94.484566], zoom_start=12)


# Add small red dots for congestion points
for idx, row in df.iterrows():
    if 'congestion_coords' in row and 'congestion_details' in row:
        for coord, detail in zip(row['congestion_coords'], row['congestion_details']):
            time, distance = detail
            folium.CircleMarker(
                location=coord,
                radius=5,  # Size of the dot
                color="red",
                fill=True,
                fill_color="red",
                fill_opacity=0.8,
                popup=f"<b>Congestion Details</b><br>Distance: {distance:.2f} miles<br>Time: {time}"
            ).add_to(congestion_map)

# Save the map with red dots
congestion_map.save("congestion_map_with_dots1.html")
"Congestion map with red dots saved as 'congestion_map_with_dots1.html'."

"Congestion map with red dots saved as 'congestion_map_with_dots1.html'."

In [ ]:
import os
import folium
import requests
import pandas as pd
from polyline import decode as decode_polyline
from pytz import timezone
from datetime import datetime
from haversine import haversine
import json

# Function to load the API key securely
def load_api_key(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            if key == 'api_key':
                return value

# Fetch route details from Google Maps Directions API
def fetch_routes(api_key, origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin,
        "destination": destination,
        "key": api_key,
        "alternatives": "true"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            routes = []
            for route in data['routes']:
                polyline = route['overview_polyline']['points']
                routes.append(decode_polyline(polyline))
            return routes
        else:
            print(f"Error from API: {data['status']}")
    else:
        print(f"HTTP Error: {response.status_code}")
    return None

# Convert UTC times to Central Time
def convert_to_central_time(utc_time_str):
    utc = timezone('UTC')
    central = timezone('US/Central')
    utc_time = datetime.strptime(utc_time_str, "%m/%d/%Y %H:%M:%S")
    central_time = utc.localize(utc_time).astimezone(central)
    return central_time.strftime("%m/%d/%Y %H:%M:%S")

# Analyze congestion flags
def update_congestion_flags(speeds_mph):
    return [speed < 20 if speed is not None else False for speed in speeds_mph]

# Find lat/long of congested points
def get_congestion_coordinates(route_polyline, distances_miles):
    route_coords = decode_polyline(route_polyline)
    congestion_coords = []
    total_distance = 0

    for i in range(1, len(route_coords)):
        dist = haversine(route_coords[i - 1], route_coords[i])
        total_distance += dist
        for distance in distances_miles:
            if total_distance >= distance:
                congestion_coords.append(route_coords[i])
                distances_miles.remove(distance)
                break
        if not distances_miles:
            break
    return congestion_coords

# Plot routes and congestion on the map
def plot_routes_and_congestion(api_key, data):
    m = folium.Map(location=[39.048786, -94.484566], zoom_start=12)
    colors = ['blue', 'green', 'purple', 'orange', 'red']

    for index, row in data.iterrows():
        origin = f"{row['origin_lat']},{row['origin_lng']}"
        destination = f"{row['destination_lat']},{row['destination_lng']}"
        routes = fetch_routes(api_key, origin, destination)

        if routes:
            for i, route_coords in enumerate(routes):
                color = colors[i % len(colors)]
                folium.PolyLine(route_coords, color=color, weight=4, opacity=0.9,
                                tooltip=f"Route {i + 1}").add_to(m)

        if 'congestion_coords' in row and 'congestion_details' in row:
            for coord, detail in zip(row['congestion_coords'], row['congestion_details']):
                time, distance = detail
                folium.CircleMarker(
                    location=coord,
                    radius=5,
                    color="red",
                    fill=True,
                    fill_color="red",
                    fill_opacity=0.8,
                    popup=f"<b>Congestion Details</b><br>Distance: {distance:.2f} miles<br>Time: {time}"
                ).add_to(m)

        # Add origin and destination markers
        folium.Marker(
            location=[row['origin_lat'], row['origin_lng']],
            popup=f"Origin: {row['origin']}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)
        folium.Marker(
            location=[row['destination_lat'], row['destination_lng']],
            popup=f"Destination: {row['destination']}",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

    return m

# Main script
if __name__ == "__main__":
    api_key_path = "/content/drive/MyDrive/API_K.txt"
    dataset_path = "/content/drive/MyDrive/chiefs-bills.xlsx"

    # Load API key
    api_key = load_api_key(api_key_path)

    # Load the dataset
    df = pd.read_excel(dataset_path)
    df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = df['query'].str.split(', ', expand=True)
    df['origin_lat'] = df['origin_lat'].astype(float)
    df['origin_lng'] = df['origin_lng'].astype(float)
    df['destination_lat'] = df['destination_lat'].astype(float)
    df['destination_lng'] = df['destination_lng'].astype(float)

 # Fetch route details (polyline) before processing congestion
    df['polyline'] = df.apply(
        lambda row: fetch_route_polyline(api_key, f"{row['origin_lat']},{row['origin_lng']}",
                                         f"{row['destination_lat']},{row['destination_lng']}"), axis=1
    )

    # Parse road timing data
    df['road_timing_parsed'] = df['road_distance_timing(meters:seconds)'].apply(
        lambda x: json.loads(x.replace("'", '"'))
    )
    df['speeds_mph'], df['congestion_flags_20mph'] = zip(
        *df['road_timing_parsed'].apply(
            lambda timing: (
                [pd.to_numeric(dist, errors='coerce') * 0.000621371 / (pd.to_numeric(time, errors='coerce') / 3600)
                 if pd.to_numeric(time, errors='coerce') > 0 else None
                 for dist, time in timing.items()],
                [pd.to_numeric(dist, errors='coerce') * 0.000621371 / (pd.to_numeric(time, errors='coerce') / 3600) < 40
                 if pd.to_numeric(time, errors='coerce') > 0 else False
                 for dist, time in timing.items()]
            )
        )
    )

    # Extract congestion details
    df['congestion_details'] = df.apply(
        lambda row: [
            (convert_to_central_time(row['datetime_utc']), pd.to_numeric(dist, errors='coerce') * 0.000621371)
            for dist, flag in zip(row['road_timing_parsed'].keys(), row['congestion_flags_20mph'])
            if flag and pd.to_numeric(dist, errors='coerce') is not None
        ],
        axis=1
    )

    # Find congestion coordinates
    df['congestion_coords'] = df.apply(
        lambda row: get_congestion_coordinates(row['polyline'], [dist for _, dist in row['congestion_details']]),
        axis=1
    )

    # Plot everything
    traffic_map = plot_routes_and_congestion(api_key, df)
    traffic_map.save("combined_map.html")
    print("Combined map saved as 'combined_map.html'")


Combined map saved as 'combined_map.html'


In [ ]:
import os
import folium
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET
from zipfile import ZipFile
from polyline import decode as decode_polyline
from pytz import timezone
from datetime import datetime
from haversine import haversine

# File paths
api_key_path = "/content/drive/MyDrive/API_K.txt"
traffic_data_file = "/content/drive/MyDrive/chiefs-bills.xlsx"
kmz_file = "/content/drive/MyDrive/inv.kmz"

# Function to load API key securely
def load_api_key(filepath):
    with open(filepath, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            if key == 'api_key':
                return value

api_key = load_api_key(api_key_path)

# Function to fetch routes from Google Maps API
def fetch_routes(api_key, origin, destination):
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {"origin": origin, "destination": destination, "key": api_key, "alternatives": "true"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            return [decode_polyline(route['overview_polyline']['points']) for route in data['routes']]
        else:
            print(f"API Error: {data['status']}")
    else:
        print(f"HTTP Error: {response.status_code}")
    return None

# Function to convert UTC time to Central Time
def convert_to_central_time(utc_time_str):
    utc = timezone('UTC')
    central = timezone('US/Central')
    utc_time = datetime.strptime(utc_time_str, "%m/%d/%Y %H:%M:%S")
    central_time = utc.localize(utc_time).astimezone(central)
    return central_time.strftime("%m/%d/%Y %H:%M:%S")

# Function to detect congestion based on speed thresholds
def update_congestion_flags(speeds_mph, threshold=40):
    return [speed is not None and speed < threshold for speed in speeds_mph]

# Function to extract congestion coordinates
def get_congestion_coordinates(polyline_str, distances):
    # Only decode the polyline if it's a string
    if isinstance(polyline_str, str):
        coordinates = polyline.decode(polyline_str)
    else:
        coordinates = polyline_str  # Use the coordinates directly if already decoded

    congestion_coords = []
    total_distance = 0
    for i, distance in enumerate(distances):
        total_distance += distance
        congestion_coords.append(coordinates[i])  # Add the coordinate corresponding to the distance

    return congestion_coords


# Extract cameras and DMS locations from the KMZ file
kml_extracted_path = "/content/drive/MyDrive/extracted_kml.kml"
with ZipFile(kmz_file, 'r') as kmz:
    kml_filename = [file for file in kmz.namelist() if file.endswith('.kml')][0]
    kmz.extract(kml_filename, path="/content/drive/MyDrive")
    kml_full_path = f"/content/drive/MyDrive/{kml_filename}"

# Parse KML file
namespace = {"kml": "http://www.opengis.net/kml/2.2"}
tree = ET.parse(kml_full_path)
root = tree.getroot()
camera_dms_data = []
for placemark in root.findall(".//kml:Placemark", namespace):
    name = placemark.find("kml:name", namespace).text
    coords = placemark.find(".//kml:coordinates", namespace).text.strip()
    lon, lat, _ = map(float, coords.split(","))
    camera_dms_data.append({"Name": name, "Latitude": lat, "Longitude": lon})
camera_dms_df = pd.DataFrame(camera_dms_data)

# Load traffic data
df = pd.read_excel(traffic_data_file)
df[['origin_lat', 'origin_lng', 'destination_lat', 'destination_lng']] = df['query'].str.split(', ', expand=True)
df['origin_lat'] = df['origin_lat'].astype(float)
df['origin_lng'] = df['origin_lng'].astype(float)
df['destination_lat'] = df['destination_lat'].astype(float)
df['destination_lng'] = df['destination_lng'].astype(float)

# Fetch route details before processing congestion
df['polyline'] = df.apply(
    lambda row: fetch_routes(api_key, f"{row['origin_lat']},{row['origin_lng']}",
                             f"{row['destination_lat']},{row['destination_lng']}"), axis=1
)

# Parse road timing data
df['road_timing_parsed'] = df['road_distance_timing(meters:seconds)'].apply(
    lambda x: json.loads(x.replace("'", '"'))
)

# Convert road timing data to speeds and detect congestion
df['speeds_mph'], df['congestion_flags'] = zip(
    *df['road_timing_parsed'].apply(
        lambda timing: (
            [pd.to_numeric(dist, errors='coerce') * 0.000621371 / (pd.to_numeric(time, errors='coerce') / 3600)
             if pd.to_numeric(time, errors='coerce') > 0 else None
             for dist, time in timing.items()],
            update_congestion_flags(
                [pd.to_numeric(dist, errors='coerce') * 0.000621371 / (pd.to_numeric(time, errors='coerce') / 3600)
                 if pd.to_numeric(time, errors='coerce') > 0 else None
                 for dist, time in timing.items()]
            )
        )
    )
)

# Extract congestion details
df['congestion_details'] = df.apply(
    lambda row: [
        (convert_to_central_time(row['datetime_utc']), pd.to_numeric(dist, errors='coerce') * 0.000621371)
        for dist, flag in zip(row['road_timing_parsed'].keys(), row['congestion_flags'])
        if flag and pd.to_numeric(dist, errors='coerce') is not None
    ],
    axis=1
)

df['congestion_coords'] = df.apply(
    lambda row: get_congestion_coordinates(row['polyline'][0], [dist for _, dist in row['congestion_details']])
    if row['polyline'] else [],
    axis=1
)

# Initialize map
traffic_map = folium.Map(location=[39.048786, -94.484566], zoom_start=12)

# Add congestion points (red dots)
for idx, row in df.iterrows():
    if row['congestion_coords']:
        for coord in row['congestion_coords']:
            folium.CircleMarker(
                location=coord,
                radius=5,
                color="red",
                fill=True,
                fill_color="red",
                fill_opacity=0.8,
                popup=f"<b>Congestion Detected</b>"
            ).add_to(traffic_map)

# Define a list of distinct colors
route_colors = [
    "blue", "green", "black", "purple", "orange", "darkred", "darkblue",
    "cadetblue", "darkpurple", "lightblue", "lightgreen", "lightgray",
   "pink", "beige", "lightred"
]

# Plot traffic routes (multiple alternatives)
for idx, row in df.iterrows():
    if row['polyline']:  # Ensure we have routes
        for i, route in enumerate(row['polyline']):  # Iterate through all alternative routes
            color = route_colors[i % len(route_colors)]  # Cycle through colors

            folium.PolyLine(
                locations=route,
                color=color,
                weight=3,
                opacity=0.8,
                tooltip=f"Route {i + 1} from {row['origin']} to {row['destination']}"
            ).add_to(traffic_map)



# Add traffic routes (blue)
for _, row in df.iterrows():
    if row['polyline']:
        folium.PolyLine(
            row['polyline'][0],
            color="blue",
            weight=3,
            opacity=0.8
        ).add_to(traffic_map)



# Plot Cameras (CCTV) - Blue Camera Icons
for _, row in camera_df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=f"Camera: {row['Name']}",
        icon=folium.Icon(color="blue", icon="camera"),
    ).add_to(traffic_map)

# Plot DMS (Message Signs) - Green Info Icons
for _, row in dms_df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=f"DMS: {row['Name']}",
        icon=folium.Icon(color="green", icon="info-sign"),
    ).add_to(traffic_map)



# Save the combined map
traffic_map.save("traffic_map_with_routes_cameras_congestion.html")
print("Map saved as 'traffic_map_with_routes_cameras_congestion.html'")


Map saved as 'traffic_map_with_routes_cameras_congestion.html'
